# Simulate a Noise map and read a CMB sim
Need to be equal noise and cmb simulations. Takes ages to simulate one for me so if you have a few or have time, this will give good results with like 20 noise and CMB sims.

In [ ]:
import sim_prep
import Beam_Profiles
noise_map = sim_prep.simnoise('LT2_MFF2')
cmb_map, cmb_constants = sim_prep.read_cmb_maps('LT2_MFF2', 1)

# Catalogue Data

In [ ]:
import pandas as pd
import numpy as np
full_df_cut = pd.read_csv('irsa_catalog_search_results_tbl_cut.csv')
df_cut = full_df_cut[full_df_cut['flux1'].notna()]
df_cut = df_cut[df_cut['flux2'].notna()]
df_cut = df_cut[df_cut['flux3'].notna()]
df_cut = df_cut[df_cut['flux4'].notna()]
df_cut = df_cut[df_cut['flux5'].notna()]
df_cut = df_cut[df_cut['flux6'].notna()]

sorted_flux_cut = df_cut.sort_values(['flux6'], ascending = [False])
sorted_flux_cut = sorted_flux_cut.reset_index(drop = True)
g_lons_cut = sorted_flux_cut[sorted_flux_cut.columns[1]]
g_lats_cut = sorted_flux_cut[sorted_flux_cut.columns[2]]

fluxvalues = {
    'flux1': '30 GHz',
    'flux2': '44 GHz',
    'flux3': '70 GHz',
    'flux4': '100 GHz',
    'flux5': '143 GHz',
    'flux6': '217 GHz',
    'flux7': '353 GHz'
}

names = sorted_flux_cut['name'][[0]]
mapglons = np.array(g_lons_cut[[0]])
mapglats = np.array(g_lats_cut[[0]])
mapflux143 = np.array(sorted_flux_cut['flux4'][[0]])


# Mask and add source
#### Mask map around source coordinates, add sorce and convolve with 2D Gaussian beam

In [ ]:
beam_arcm, cmb_nside, freq, channel = cmb_constants
flux2use = mapflux143

beam_maps = beam_cutout(cmb_map, [np.array(noise_map)], beam_arcm, flux2use, 
                   (mapglons, mapglats), freq, 
                   cmb_nside = 4096, shape_out = (512, 512))

# Calculate B_ell from radial profiles

In [ ]:
radial_profiles, all_bls, mean_bl, confidence_intervals, pivot_ell = beam_profile(beam_maps, beam_arcm, flux2use)

# Calculate Eigenmodes

In [ ]:
columns = map(str, range(len(cmb_map)))
flux = 0
edict = pd.DataFrame(columns = columns)
for i in np.arange(len(cmb_map)):
    edict[str(i)] = testbls[flux][i]
evals, evecs = utils.emodes(edict)

In [ ]:
# Take largest 4 eigenmodes only to plot or use in 2 delta B_ell / B_ell
for i in range(len(evecs[flux][0]) - 4, len(evecs[flux][0])):
    emode_vec = []
    for j in range(len(evecs[flux])):
        emode_vec.append(all_evecs[flux][j][i])
    emode = np.array(emode_vec) * np.sqrt(evals[flux][i])